In [1]:
import sys
sys.path.append('../src/')

import pandas as pd
import requests
import numpy as np
import geocoder
import data_io
import time
import tokens

import geocode_utils as gutils
import utils

ModuleNotFoundError: No module named 'geocode_utils'

In [3]:
import imp
imp.reload(tokens)

<module 'tokens' from '../src/tokens.py'>

In [14]:
#locations that failed to geocode but could be filled by hand
WEIRD_LOC_DICT = {'westseneca':'west seneca, ny',
                 'linolakes': 'lino lakes, mn',
                 'newportrichey':'new port richey, fl',
                 'mayslick': 'mays lick, ky',
                 'boiseid': 'boise, id',
                 'dixonil': 'dixon, il',
                 'dallastx': 'dallas, tx',
                 '5779purpleleafcourtfrederickmd': 'frederick, md',
                 '07060northplainfieldnj': 'north plainfield, nj',
                 '08108haddontownshipnj': 'haddon township, nj',
                 '33134coralgablesfl': 'coral gables, fl',
                 '92587canyonlakeca': 'canyon lake, ca',
                 '95682cameronparkca': 'cameron park, ca',
                 '92075solanabeachca': 'solana beach, ca',
                 'de forest, wi': 'deforest, wi',
                 'horsesho, tx': 'horseshoe, tx',
                 'wesley c, fl': 'wesley chapel, fl'}

CENSUS_KEY = tokens.CENSUS_KEY
MB_TOKENS = tokens.MAPBOX_TOKENS

STATE_DICT = dict((v,k) for k,v in utils.us_state_abbrev.items())
CLEANED_FOLDER_NAME = 'mapbox_geocode'
NOT_FOUND_FILENAME = 'locs_not_found_master_recode_v2.csv'

In [15]:
import string
def contains_only_digits(s):
    # True for "", "0", "123"
    # False for "1.2", "1,2", "-1", "a", "a1"
    if type(s) != str:
        return True
    for ch in s:
        if not ch in string.digits:
            return False
    return True    

def has_numbers(inputString):
     return any(char.isdigit() for char in inputString)


def clean_location(x, other_loc, state):
    if contains_only_digits(other_loc) == False:
        if type(state) == str:
            state = state.lower()
        for k in WEIRD_LOC_DICT.keys():
            if k in x:
                x = WEIRD_LOC_DICT[k]
                return x
        if has_numbers(x) == True:
            x = ''.join(c for c in x if not c.isnumeric())
            if ' - ' in x:
                x = x.replace(" - ", "")
            if '-' in x:
                x = x.replace("-", "")
        if x[-4:] == ', us':
            x = x[0:len(x)-4]
        if ',' not in x and type(state)==str:
            comma_loc = x.find(state)
            old = x
            start = old[0:comma_loc]
            print(old, other_loc, state)
            try:
                if start[-1] == ' ':
                    start = start[0:len(start)-1]
                x = start +', '+old[comma_loc:]
                print(x)
            except:
                x = 'none'
            
        if 'a f b' in x:
            x = x.replace('a f b', 'air force base')
        if 'mc ' in x:
            if 'mc ,' not in x:
                x = x.replace('mc ', 'mc')
        if x[0]==' ':
            x = x[1:]
        
        
    return x

def clean_other_loc(x, other_loc, state):
    if contains_only_digits(other_loc) == False:
        if type(x) == str:
            if type(state) == str:

                state = state.lower()

            for k in WEIRD_LOC_DICT.keys():
                if k in x:
                    x = WEIRD_LOC_DICT[k]
            if has_numbers(x) == True:
                x = ''.join(c for c in x if not c.isnumeric())
                if ' - ' in x:
                    x = x.replace(" - ", "")
                if '-' in x:
                    x = x.replace("-", "")
            if x[-4:] == ', us':
                x = x[0:len(x)-4]
            if ',' not in x and type(state)==str:
                comma_loc = x.find(state)
                old = x
                start = old[0:comma_loc]
                print(old, other_loc, state)
                try:
                    if start[-1] == ' ':
                        start = start[0:len(start)-1]
                    x = start +', '+old[comma_loc:]
                    print(x)
                except:
                    x = 'none'

            if 'a f b' in x:
                x = x.replace('a f b', 'air force base')
            if 'mc ' in x:
                if 'mc ,' not in x:
                    x = x.replace('mc ', 'mc')
            if x[0]==' ':
                x = x[1:]
            
            x = x[0:len(x)-2]
        
        
    return x


In [102]:
def get_token(index):
    tokens =  MB_TOKENS
    if index % 2 == 0:
        return tokens[0]
    else:
        return tokens[1]
    
'''
GEOCODER FUNCTION
'''

def __coords_to_fip(lat,lng):
    result_d = {'search_status':'',
             'state_county_fips_str':np.nan,
             'county_name':np.nan}
    try:
        #get FIP code for corresponding lat/long
        fcc_req = 'https://geo.fcc.gov/api/census/block/find?'\
        +f'latitude={lat}&longitude={lng}&showall=false&format=json&key={CENSUS_KEY}'
        resp = requests.get(fcc_req)
        result = resp.json()
    except Exception as e:
        msg = 'error in fip retrieval: ' + str(e)
        print(msg)
        result_d['search_status'] = result_d['search_status'] + msg + '; '

    try:
        state_county_fips = result['County']['FIPS']
        county = result['County']['name']
        result_d['state_county_fips_str'] = state_county_fips
        result_d['county_name'] = county
    except Exception as e:
        msg = 'error in fip retrieval: ' + str(e)
        result_d['search_status'] = result_d['search_status'] + msg 
    
    return result_d
                            
def assign_county_fips(df, min_index = 0):
    start_time = time.time()
    #not_found_df = pd.DataFrame()
    
    for i,(idx,row) in enumerate(df.iterrows()):
        if not pd.isnull(row.state_county_fips_str): 
            # this row is geocoded already
            continue 
        time.sleep(0.5)
        token = get_token(i)
        location = row['location_tosearch']
        state_full = row['location_statefullname']
        
        print(location)
        
        # limit search results to US districts,place and locality
        # this is very helpful so MapBox doesnt match with the wrong place
        g = geocoder.mapbox(location, key=token, country='US',types='district,place,locality')
        df.loc[idx,'search_status'] = g.status
        
        g = g.json
        
        if g is None:
            print('nothing returned!')
        else:
            if g['status'] == 'OK':
                if 'state' in g.keys():
                    if g['state'] == state_full and g['country'] == 'United States':
                        lat = g['lat']
                        lng = g['lng']
                        df.loc[idx, 'location_lat'] = lat
                        df.loc[idx, 'location_lng'] = lng
                        result_d = __coords_to_fip(lat,lng)
                        for k,v in result_d.items(): df.loc[idx,k] = v
            else:
                print('status issue: ', g)

        if i % 100 == 0 and i > min_index:
            print(i)
            curr_time = time.time()
            print('min since start: ', (curr_time-start_time)/60)
            print('saving...')
            df.to_csv(data_io.input_cleaned/'geolocations'/f'unique_locations_w_fips_scraping.csv',
                                     encoding='utf-8')
    return df
    
    

In [82]:
df = pd.read_csv(data_io.input_cleaned/'geolocations'/'unique_locations_to_scrape.csv',index_col=[0])

Construct column to be used for searching, only for first time searching

In [83]:
rescrape = True
if not rescrape:
    df['location_statefullname'] = df.location_stateprefix.map(STATE_DICT)
    print(df.location_statefullname.isna().sum())
    # prefer searching with city, fullstate_name over city, state_abbreviation
    df['location_tosearch'] = df.location_city.where(df.location_statefullname.isna(),
                                            df.location_city_only + ', ' + df.location_statefullname)

    df['state_county_fips_str'] = np.nan
    df['county_name'] = np.nan
    df.loc['search_status'] = ''

0


In [86]:
# df['cleaned_location_city'] = df.apply(
#     lambda x: clean_location(x['location_city'], x['location_city_only'], x['location_stateprefix']), axis = 1)

# df['cleaned_location_city_only'] = df.apply(
#     lambda x: clean_other_loc(x['location_city'], x['location_city_only'], x['location_stateprefix']), axis = 1)


In [103]:
outputs = assign_county_fips(df)
# writer = pd.ExcelWriter(data_io.input_cleaned/'geolocations'/'cancer_unique_locs_w_fips_mapbox.xlsx',
#                        engine='xlsxwriter', options = {'strings_to_urls':False, 'strings_to_formulas':False})
# outputs.to_excel(writer, index=False, encoding='utf-8')
# writer.close()

Bucksville, Alabama
Broyles Island, Tennessee
Indian Service Rt 210, New Mexico
Western Hills, Colorado
Meadowbrook, Alabama
Waimea/Kamuela Cdp, Hawaii


Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Waimea/Kamuela Cdp, Hawaii.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Waimea/Kamuela%20Cdp,%20Hawaii.json?access_token=pk.eyJ1IjoiaHF0cnVvbmcxMSIsImEiOiJja240eDZjdDAxbHkxMnZtaDJpODJzbGthIn0.tD6DehvVJVsDRP2A2VM8dw&country=US&types=district%2Cplace%2Clocality


nothing returned!
Benton Hills, North Carolina
Foothill Farms, California
Chester, Washington
Forestville, Ohio
Catlett, California
Amwell Township, Pennsylvania
Berea, Virginia
Brookhaven, South Carolina
Main Street, Connecticut
Paradise, Nevada
Roosterville, Missouri
Douglas Hill, Maine
Qualla, North Carolina
Green Valley, Maryland
Desmoines, Washington
Arden, Nevada
Blue Mount, Maryland
Bailey S Crossroads, Virginia
Greenbriar Heights, Virginia
River Glen, Connecticut
Chewnings Corner, Virginia
Oswichee, Alabama
Quiet Dell, West Virginia
Lato, Alabama
Orangeville, Michigan
Alston Park, California
Federal Hills, Georgia
Berkshire, Ohio
Alton, Ohio
Creesville, New Jersey
Aberdeen, Florida
Buckeye, Washington
KuróWka, Maryland
Acy, Louisiana
Shannon Hill, Virginia
Baywalker Village, Maryland
Morristown, Vermont
Berkshire, Connecticut
Coffing, California
Richfield, New York
Arden Town, California
Bleecker, Alabama
Matamoras, Ohio
South Allapattah, Florida
Lakeland South, Washington
Lon

Deale, Maryland
Waverly, Alabama
Rumbels, Pennsylvania
Bear Creek, Wisconsin
Swoope, Virginia
Connelly, New York
Rockford, Washington
Hancock, Minnesota
Nicholson, Pennsylvania
Swanzey, New Hampshire
Bulls Head, New York
Shorter, Alabama
Parmalee, Florida
Pinnacle, North Carolina
Campbell Island, New York
Lower Allen, Pennsylvania
Girdner, Missouri
Hope, Michigan
Cedar Ridge, California
Barker, Texas
Belleview, Virginia
Auberry, California
Soldier, Iowa
Americus, Kansas
Claiborne, Maryland
Canyon, California
Bowdoin Center, Maine
700
min since start:  5.6841175635655725
saving...
Chico, Texas
City Of Benson, Minnesota
Bellefontaine, Missouri
Broaddus, Texas
Corridor North, Maryland
Murray, New York
Greenup, Illinois
Fleetwood, North Carolina
Chrisman, Illinois
Nesquehoning, Pennsylvania
Yosemite, Kentucky
Frazeysburg, Ohio
Edmore, Michigan
Lost Island, Tennessee
Markleysburg, Pennsylvania
Long Hammock, Florida
Oak Run, California
Randolph, Mississippi
Fulford, North Carolina
Browns Cor

Union, West Virginia
Petrolia, Texas
Belmont, Texas
Woodburn, Indiana
Banoak, North Carolina
Brookfield, Vermont
Somerville, Tennessee
Waynesville, Ohio
Keysville, Georgia
Cost, Texas
Ransom Canyon, Texas
Greenbush, Virginia
Briar Park, Delaware
Kawkawlin Township, Michigan
Weston, West Virginia
Otisco, New York
Watertown, Tennessee
West Augusta, Virginia
Robert Lee, Texas
Gurley, Nebraska
Mehoopany, Pennsylvania
Greenfield, New York
Waterloo, Nebraska
Elba, New York
Enfield, Maine
Foster, Missouri
Warm Springs, Oregon
Woodsfield, Ohio
Roodhouse, Illinois
Rosalie, Nebraska
Cobb, Oregon
Charter Township Of Ypsilanti, Michigan
Fairfield, Idaho
Goreville, Illinois
Vansant, Virginia
Mountain, Wisconsin
Meadowview, Virginia
1100
min since start:  11.389950478076935
saving...
Shirleysburg, Pennsylvania
Chamberlain, South Dakota
Fairview Heights, Illinois
Panama, Oklahoma
Leonards, New Jersey
Eben Junction, Michigan
Mcmurray, Pennsylvania
Bertrand, Missouri
Cottonwood, Minnesota
Hitchcock, So

Edwards, Mississippi
Mountain City, Georgia
Wallace, South Carolina
Moapa Valley, Nevada
Swan, Indiana
Goshen, Virginia
Alberta, Virginia
East Riverdale, Maryland
Riner, Virginia
Shorewood-Tower Hills-Harbert, Michigan
Grant Park, Illinois
Green Bluff, Washington
Urbana, Iowa
Dyke, Georgia
KalāHeo, Hawaii
Georgetown, Maine
Sullivan, Ohio
Cambridge, Illinois
Lodi, Ohio
Fort Loramie, Ohio
Fountain Hill, Arkansas
Oaks, Oklahoma
Ruby, South Carolina
Addis, Louisiana
Wales, Alaska
Palermo, Maine
San Luis, Colorado
Lennox, South Dakota
Pewee Valley, Kentucky
Red Oak, Oklahoma
Barnegat Light, New Jersey
Three Oaks, Florida
Sun River, Montana
Garrison, Kentucky
Algoma, Wisconsin
Bode, Iowa
Pleasant Grove, Maryland
Bay Mills, Michigan
1500
min since start:  16.963048978646597
saving...
Lubeck, West Virginia
Opa-Locka, Florida
Canal Fulton, Ohio
Livingston, Tennessee
Tiskilwa, Illinois
Dutton, Alabama
Spiro, Oklahoma
Piercy, California
Fredonia, Arizona
Lima, New York
Queenstown, Maryland
Long 

Hemlock, Virginia
Washington, Louisiana
Floyd Springs, Georgia
Wesson, Mississippi
Lonaconing, Maryland
Au Train, Michigan
Deerfield, Michigan
Shaw, Mississippi
South Boardman, Michigan
Clover Hill, Ohio
Cruger, Mississippi
Pender, Nebraska
Mayo, Florida
Mount Calvary, Wisconsin
Pembroke, Virginia
Caneyville, Kentucky
Groom Corners, New York
Jenkinsville, South Carolina
Weatherby, Missouri
Gillett, Texas
Volga, South Dakota
Westphalia, Michigan
Fenton Town, New York
Needmore, Tennessee
Surry, Maine
Milan, Ohio
Hartland Township, Michigan
Prospect, Connecticut
Seymour, Iowa
Geneva, Nebraska
South Wales, New York
Monroe Center, Illinois
Vernon Center, New York
Vardaman, Mississippi
Oak Ridge, Louisiana
Chireno, Texas
Hargill, Texas
Healdton, Oklahoma
Depew, Oklahoma
Beekmantown, New York
Wapello, Iowa
Beaverdam Lake-Salisbury Mills, New York
Buxton, Mississippi
Coffeeville, Mississippi
1900
min since start:  22.534182278315225
saving...
Malta, Illinois
Ellsworth, Kansas
Randall, Minnesot

Status code 403 from https://api.mapbox.com/geocoding/v5/mapbox.places/Coaldale;Six Mile Run Post Office (Historical), Pennsylvania.json: ERROR - 403 Client Error: Forbidden for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Coaldale;Six%20Mile%20Run%20Post%20Office%20(Historical),%20Pennsylvania.json?access_token=pk.eyJ1IjoiaHF0cnVvbmcxMSIsImEiOiJja240eDZjdDAxbHkxMnZtaDJpODJzbGthIn0.tD6DehvVJVsDRP2A2VM8dw&country=US&types=district%2Cplace%2Clocality


Coaldale;Six Mile Run Post Office (Historical), Pennsylvania
nothing returned!
Sylvester, West Virginia
Montezuma, Georgia
Golden, Illinois
Ruleville, Mississippi
Attica, New York
Hansville, Washington
Harpersville, Alabama
Warrenville, Connecticut
Lamont, Florida
Kelleyland, Maine
Aberdeen, Idaho
Sandy Spring, Maryland
Oconto Falls, Wisconsin
Chiles, Kentucky
Roxbury, Vermont
Florence Hill, New York
Creighton, Missouri
Glenvil, Nebraska
Forest City, Missouri
South Fulton, Tennessee
Bridgewater, Virginia
Percy, Illinois
Gardiner, New York
Franks Corner, Ohio
Collins, Georgia
Syracuse, Nebraska
Rockham, South Dakota
Great Cacapon, West Virginia
North Bend, Ohio
Babb, Montana
Audubon, Pennsylvania
Kidron, Ohio
Salemburg, North Carolina
Elrama, Pennsylvania
Boles, Missouri
Alfred Station, New York
Tilden, Nebraska
Honalo Cdp, Hawaii
Marble Falls, Arkansas
Greenback, Tennessee
Brooten, Minnesota
Thornton, Arkansas
Mineral Point, Missouri
Honalo, Hawaii
Morganza, Louisiana
Farmington, Ohio


In [104]:
outputs.to_csv(data_io.input_cleaned/'geolocations'/f'unique_locations_w_fips_scraped.csv',
                                     encoding='utf-8')

In [105]:
outputs

,location_city,location_city_only,location_stateprefix,location_statefullname,location_tosearch,state_county_fips_str,county_name,search_status,location_lat,location_lng
campaign_id,,,,,,,,,,
joyce-catozellagreentreatment,"Mykawa, TX",Mykawa,TX,Texas,"Mykawa, Texas",48167,Galveston,,29.383800,-94.90270
rueqt4-the-kelley-family,"Lakeside Park, KY",Lakeside Park,KY,Kentucky,"Lakeside Park, Kentucky",21117,Kenton,,39.035600,-84.56910
a4a3kq-help-chris-fight-cancer,"Summit, IL",Summit,IL,Illinois,"Summit, Illinois",17031,Cook,,41.788100,-87.81030
cecily-journey-to-healing,"Pamela Heights, TX",Pamela Heights,TX,Texas,"Pamela Heights, Texas",48201,Harris,,29.810000,-95.43000
help-fallons-thyroidectomy-journey,"Washington Township, PA",Washington Township,PA,Pennsylvania,"Washington Township, Pennsylvania",42125,Washington,,40.174000,-80.24620
...,...,...,...,...,...,...,...,...,...,...
p37m8k-team-bailey,"Powderly, KY",Powderly,KY,Kentucky,"Powderly, Kentucky",21177,Muhlenberg,,37.237000,-87.15920
supporting-cindy-goff,"Mehama, OR",Mehama,OR,Oregon,"Mehama, Oregon",41047,Marion,,44.790281,-122.61806
help-susan-and-kevin-survive,"Bentley Springs, MD",Bentley Springs,MD,Maryland,"Bentley Springs, Maryland",24005,Baltimore,,39.673700,-76.67080


In [ ]:
token = get_token(0)
g = geocoder.mapbox('Washington Township, Pennsylvania', key=token,country='US',types='district,place,locality')
g = g.json
for k in g.keys():
    print(k,g[k])